In [1]:
import os

import pandas as pd

# 1. Read all CSV files into a single data frame

In [2]:
# Identify the location of the original files
# This represents the path: ../data/waitrose-2024-07
data_folder = os.path.join('..', 'data', 'waitrose-2024-07')

# Use a list comprehension to get all the files in the folder
all_files = [os.path.join(data_folder, file) for file in os.listdir(data_folder) 
             if file.endswith('.csv')]

# Print the list of files if you want to check
# print(all_files)

# Read every single file as a DataFrame
# Save the dataframes in a list
list_of_dfs = [pd.read_csv(file) for file in all_files]

# Use pd.concat to concatenate all the files into a single DataFrame
df = pd.concat(list_of_dfs)

# Check that we have all the data
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25418 entries, 0 to 1593
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data-product-id        25418 non-null  int64  
 1   data-product-name      25418 non-null  object 
 2   data-product-type      25418 non-null  object 
 3   data-product-on-offer  25418 non-null  bool   
 4   data-product-index     25408 non-null  float64
 5   image-url              25418 non-null  object 
 6   product-page           25418 non-null  object 
 7   product-name           25407 non-null  object 
 8   product-size           25363 non-null  object 
 9   item-price             25407 non-null  object 
 10  price-per-unit         24976 non-null  object 
 11  offer-description      7201 non-null   object 
 12  category               25418 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(10)
memory usage: 2.5+ MB


# 2. Exploring the `item-price` column

<div style="background-color:white;padding:0.5em;margin-left:0em;margin-bottom:1em;border-radius:0.5em;font-family: monospace;border: 1px solid #eda291;font-size:0.9em;width:70%;line-height:1em">

**What did I learn? What are the different formatting scenarios?**

1. Some numbers are in the format `£10.00` (with the pound symbol)
    - I just need to remove the pound symbol from those 
    - 96% of the data was in this format
2. Some are in the format `75p` (indicating pences)
    - Those I need to convert to `0.75`
    - 3% of the data was in this format
3. There are cases with `each est.`

**Problems I found along the way**

- Before any pre-processing, I tried `df['item-price'].astype(float)` but, of course, it threw an error. That's how I started to discover the formatting scenarios above.

- I created a first version of the `clean_item_price()` function that removed the `£` with the method `"some_string".replace("£", "")` 
    - I learned about `replace()` from previous weeks, but I also found it on the Python's official documentation [here](https://docs.python.org/3/library/stdtypes.html#str.replace).

- But then I got an error saying that a float does not have the `replace()` method. I fixed it by forcing the `item_price` variable in the function to a string.
    - `replace()` is only defined for string objects.

</div>

In [3]:
# The data type of this column is 'O' (object)
df['item-price'].dtype

dtype('O')

⚠️ When a column (a pd.Series) is of type object, this means that pandas could not discern a specific data type (numeric types, boolean, datetime, etc) for this column. Frequently, this means the data is a string, but it could be that some objects are strings, others are numbers, others are something else. 

**To treat this column as a column of strings, I will have to convert each element to a string explicitly with the `str()` method, or the entire column with `astype(str)`**

## Validation

How do I know if the column can be converted to floats?

I used the `pandas.Series.astype(float)` function: 


```python
# This is a way to validate if my column can be converted to float
# If the column is clean, this will not throw an error
df['item-price'].astype(float)
```

```python
ValueError: could not convert string to float: '£10.00'
```

GitHub Copilot pointed me an alternative way to check for errors in the conversion using AI.

I like this one as it doesn't throw any errors:

In [5]:
# Alternatively, I could check for errors like this:

total_invalid_numbers = pd.to_numeric(df['item-price'], errors='coerce').isna().sum()
ratio_invalid_numbers = 100*(total_invalid_numbers/len(df))

print(f"{ratio_invalid_numbers:.2f}% of the column cannot be directly converted to a float.")

100.00% of the column cannot be directly converted to a float.


## A fix for the problem

In [7]:
def clean_item_price(item_price: str): 

    # Force the item_price to be a string before doing anything else
    item_price = str(item_price)

    if '£' in item_price:
        # For example: item_price = '£1.99' --> '1.99'
        new_item_price = item_price.replace('£', '')
    elif item_price.endswith('p'):
        # For example: item_price = '75p' --> '0.75p'
        new_item_price = '0.' + item_price.replace('p', '')
    else:
        # Keep it as is and hope it doesn't throw any new errors
        new_item_price = item_price
    
    return new_item_price

Checking that the function above works for individual cases:

In [8]:
clean_item_price('£1.99')

'1.99'

In [9]:
clean_item_price('75p')

'0.75'

## Validation (again)

- Use the `apply()` function to modify all the elements of the column.
- Save the output to a separate Series
- Check how many elements of that new Series is now free of errors when converting to floats

In [11]:
# Alternatively, I could check for errors like this:

new_item_price = df['item-price'].apply(clean_item_price)

total_invalid_numbers = pd.to_numeric(new_item_price, errors='coerce').isna().sum()
ratio_invalid_numbers = 100*(total_invalid_numbers/len(df))

print(f"{ratio_invalid_numbers:.2f}% of the column cannot be directly converted to a float.")

1.70% of the column cannot be directly converted to a float.


**Success!** Nearly 98% of the data has been clean now!

## TODO

What are the remaining cases?

In [12]:
# Returns a boolean Series (just Trues and Falses)
# Whenever the output is True: 
#   it means that when pandas tried to convert it to a numeric, it saw a NaN (not a number)
# Whenever the output is False:
#   the clean_item_price() function removed the non-numerical characters
#   and pandas was able to convert the remaining string to a numerical without an issue
problematic_rows = pd.to_numeric(new_item_price, errors='coerce').isna()

# See how this looks like
problematic_rows

0       False
1       False
2       False
3       False
4       False
        ...  
1589     True
1590    False
1591    False
1592    False
1593    False
Name: item-price, Length: 25418, dtype: bool

How does df['item-price'] looks like for cases where `problematic_rows == True`?  

In [16]:
# Adding each est. to the list of scenarios
df['item-price'][problematic_rows == True].head(20)

1615       32p each est.
33       £4.76 each est.
39         10p each est.
65       £4.90 each est.
73       £5.99 each est.
95      £14.22 each est.
101      £6.79 each est.
122      £5.64 each est.
132      £7.99 each est.
135      £2.84 each est.
137     £10.29 each est.
142      £8.59 each est.
158      £4.82 each est.
167      £6.67 each est.
180      £6.16 each est.
193      £4.00 each est.
194      £7.85 each est.
195     £14.95 each est.
200      £6.28 each est.
202      £6.67 each est.
Name: item-price, dtype: object